# Welcome to Learning Enabled Optimization(LEO) Software

## What does this software do?

The goal of this software is to allow users to create **statistical learning (SL)** models from data, and then use these models within **optimization models besed on stochastic programming(SP)**. As part of this process, we also support **model validation and selection**.

## What will you need to do to use LEO?

### 0. New Project:

 #### 0.1 Generate a new project.

#### 0.2 Generate a log file(.txt).

### 1. SL data:

#### 1.1 The SL training dataset should be processed using SL models available to the user (e.g. R, Matlab).

#### 1.2 After processing the data in 1.1, click menu to input SL data and error terms.

#### 1.3 Once step 1.2 in completed, LEO software will perform a χ2-test to identify outliers.

### 2. SP model: choose one of the following:

#### 2.1 Use PySP to generate input files for Stochastic Decomposition(SD)

#### 2.2 Create SMPS input files and then run the SD solver within LEO

#### 2.3 Use some other SP solver which provides the first-stage decision and the second-stage cost-to-go (recourse) function


### 3. Validation data

#### 3.1 Import training and validation cost-to-go dataset(In the Model Validation Menu).

#### 3.2 Do T-test, F-test, and χ2-test (the outputs are shown in the log_data_time.txt).

#### 3.3 Import all the cost-to-go output files for validation data generated by all models(In the Compare Menu).

#### 3.4 Do Kruskal-Wallis Test(the outputs are shown in the log_data_time.txt). Draw the stochastic dominance figure to compare different models.

The process is summarized in this figure: